In [41]:
modelpath='자연어처리 모델.h5'
naverdatapath="../데이터 저장소/NaverNew_labeling105.csv"
word=train.iloc[:1]

In [42]:
import pandas as pd
from konlpy.tag import Mecab
from collections import Counter
from collections import defaultdict
from tqdm import tqdm
from konlpy.tag import Komoran
from sklearn.model_selection import train_test_split
komoran = Komoran()

mecab = Mecab("C:\\mecab\\mecab-ko-dic")

word['내용'] = word['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한',
             '에', '하', '고', '을', '를', '인', '듯', '과', 
             '와', '네', '들', '듯', '지', '임', '게', '만', 
             '게임', '겜', '되', '음', '면','있','으로','했','로', '할']
cols=['inflation' , #인플레이션
    'treasury_bonds' , #국채
    'tightening' , #긴축
    'ormality' , #정상
    'powell' , #파월
    'dispute' , #분쟁
    'japan' , #일본
    'volume' , #물량
    'chairman' , #의장
    'remarks' , #발언
    'thought' , #사상
    'effect' , #효과
    'anxiety' , #불안
    'buying' , #매수세
    'volatility' , #유동
    'early_stage' , #초반
    'decline' , #낙폭
    'learning_result'] #학습결과
df=pd.DataFrame()
wordlist=['인플레이션', '국채', '긴축', '정상', '파월', '분쟁'
              , '일본', '물량','의장','발언','사상','효과','불안','매수세','유동','초반','낙폭','date']
list1=[[]for _ in range(17)]
date=[0]
text=train["내용"][0]
text = mecab.nouns(text)
text = [item for item in text if item not in stopwords]
text = np.hstack(text)
word_count = Counter(text)
for y in range(17):
    list1[y].append(word_count[wordlist[y]])

for x in range(17):
    df[cols[x]]=list1[x] 
df["date"]=word["Date"]
word["내용"]
word["내용"]=pd.Series(map(lambda x:" ".join(mecab.nouns(x)) ,word["내용"]))

#데이터 불러오기
train=pd.read_csv(naverdatapath)
train=train.dropna(axis=0)
train=train.reset_index(drop=True)
X_m_t=train["내용"]
Y_m_t=train["label"]
x_train_m, x_val_m, y_train_m, y_val_m=train_test_split(X_m_t,Y_m_t,test_size=0.2,random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

NGRAM_RANGE = (1, 4) # 주변 요소 묶어서 분석 (1,4)->4개까지 묶어서 분석
TOP_K = 20000 # 최대 토큰 개수
TOKEN_MODE = 'word' #word 단어단위분석 char 문자단위분석
MIN_DOCUMENT_FREQUENCY = 4 #뭐지?

def ngram_vectorize(train_texts, train_labels, val_texts):

    kwargs = {
            'ngram_range': NGRAM_RANGE,  
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE, 
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)
    x_train = vectorizer.fit_transform(train_texts)
    x_val = vectorizer.transform(val_texts)
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

X_m_t=word["내용"]
NGRAM_RANGE = (1, 4)
TOKEN_MODE = 'word'
MIN_DOCUMENT_FREQUENCY = 2
TOP_K = 20000
x_train_m_ngr,x_test_m_ngr=ngram_vectorize(x_train_m,y_train_m,X_m_t)

from keras.models import load_model
model = load_model(modelpath)
pred=model.predict(x_test_m_ngr)
df[cols[17]]=pd.Series(pred[0])

C:\Users\DM\AppData\Local\Temp\ipykernel_29748\2044020010.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  word['내용'] = word['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\DM\AppData\Local\Temp\ipykernel_29748\2044020010.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word['내용'] = word['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\DM\AppData\Local\Temp\ipykernel_29748\2044020010.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word["내용"]=pd.Ser

1/1 [==============================] - 0s 71ms/step


In [46]:
df

,inflation,treasury_bonds,tightening,ormality,powell,dispute,japan,volume,chairman,remarks,thought,effect,anxiety,buying,volatility,early_stage,decline,date,learning_result
0,0,0,0,1,0,0,8,1,1,0,0,14,0,2,1,0,3,2016-01-04,0.493398
